In [18]:
%matplotlib inline

import importlib
import itertools
import numpy as np
import matplotlib.pyplot as plt
import hwutils as utils

### Generalization Error

**Problem 1**

In [12]:
class generalization_error_bound:
    
    def __init__(self, VC_dimension, delta):
        self.growth = lambda N: N ** VC_dimension  # growth function approximate bound
        self.delta = delta
    
    def VC(self, N):
        return np.sqrt(8 / N * np.log(4 * self.growth(2*N) / self.delta))
    
    def Rademacher_penalty(self, N):
        return np.sqrt(2 / N * np.log(2 * N * self.growth(N))) \
             + np.sqrt(2 / N * np.log(1 / self.delta)) \
             + 1 / N

In [17]:
m_approx = get_growth_func_approx_bound(10)
delta = 1 - 0.95
epsilon_max = 0.05

for N in itertools.count(start=1):
    if get_epsilon(delta, N, m_approx) <= epsilon_max:
        print("VC generalization bound sample size prediction: {0}".format(N))
        break

VC generalization bound sample size prediction: 452957


**Problem 2**